In [1]:
import sys
import os
from pathlib import Path
import pandas as pd
import datetime as dt
from mlrun.platforms import auto_mount


DATA_DIR = "data"
MLRUN_ENV_FILE = "mlrun.env"

import mlrun

mlrun.set_env_from_file(MLRUN_ENV_FILE)

project = mlrun.get_or_create_project(
    "predictive-maintenance",
    parameters={
        "source": "s3://mlrun/predictive-maintenance.zip",
        "default_image": "felipenv/mlrun-predictive:1.9",
    },
)

project.set_secrets(
    secrets={
        "PG_HOST": "optimus-postgres.mlrun.svc.cluster.local",
        "PG_USER": "optimus",
        "PG_PASSWORD": "optimus",
        "PG_DB": "optimus",
    }
)

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


> 2025-09-02 15:33:30,622 [info] Loading project from path: {"path":"./","project_name":"predictive-maintenance","user_project":false}
Project Source: s3://mlrun/predictive-maintenance.zip
Exporting project as zip archive to s3://mlrun/predictive-maintenance.zip...
> 2025-09-02 15:33:34,054 [info] Project loaded successfully: {"path":"./","project_name":"predictive-maintenance","stored_in_db":true}


## 0. Log Datasets + Configs

In [2]:
from datasetsforecast.phm2008 import PHM2008

In [3]:
Y_train_df, Y_test_df = PHM2008.load(directory='./data', group='FD001', clip_rul=False)


In [4]:
Y_train_df = project.log_dataset(
    key="Y_train_df", df=Y_train_df, format="csv"
)

Y_test_df = project.log_dataset(
    key="Y_test_df", df=Y_test_df, format="csv"
)

## 1. Preprocessing Function

In [5]:
preprocessing_fn = project.get_function("preprocessing")

# preprocessing_fn.spec.image_pull_policy = "Always"

In [6]:
preprocessing_run = project.run_function(
    function=preprocessing_fn,
    inputs={
        "Y_train_df": Y_train_df.uri,
        "Y_test_df": Y_test_df.uri,
    },
    # returns=[
    #     {"key": "post_enforce_schema", "file_format": "csv"},
    #     {"key": "resampled_data", "file_format": "csv"},
    #     ],
    local=False,
)

> 2025-09-02 15:33:41,774 [info] Storing function: {"db":"http://mlrun-api:8080","name":"preprocessing-input-data","uid":"b6f05fa4c86e468393d838047d64f1bb"}
> 2025-09-02 15:33:42,277 [info] Job is running in the background, pod: preprocessing-input-data-w9fjt
> 2025-09-02 15:33:49,370 [info] extracting source from s3://mlrun/predictive-maintenance.zip to /app/code
> 2025-09-02 15:33:51,143 [info] Found 14 sensor columns: ['s_2', 's_3', 's_4', 's_7', 's_8', 's_9', 's_11', 's_12', 's_13', 's_14', 's_15', 's_17', 's_20', 's_21']
> 2025-09-02 15:33:51,849 [info] Globally normalized 14 sensor columns
> 2025-09-02 15:33:52,252 [info] To track results use the CLI: {"info_cmd":"mlrun get run b6f05fa4c86e468393d838047d64f1bb -p predictive-maintenance","logs_cmd":"mlrun logs b6f05fa4c86e468393d838047d64f1bb -p predictive-maintenance"}
> 2025-09-02 15:33:52,253 [info] Run execution finished: {"name":"preprocessing-input-data","status":"completed"}


project,uid,iter,start,end,state,kind,name,labels,inputs,parameters,results,artifacts
predictive-maintenance,...7d64f1bb,0,Sep 02 15:33:49,2025-09-02 15:33:52.239005+00:00,completed,run,preprocessing-input-data,kind=jobowner=jovyanmlrun/client_version=1.9.2mlrun/client_python_version=3.11.13host=preprocessing-input-data-w9fjt,Y_train_dfY_test_df,,"return=( Unnamed: 0 unique_id ds s_2 ... s_17 s_20 s_21 y\n0 0 1 1 -1.721725 ... -0.781710 1.348493 1.194427 191\n1 1 1 2 -1.061780 ... -0.781710 1.016528 1.236922 190\n2 2 1 3 -0.661813 ... -2.073094 0.739891 0.503423 189\n3 3 1 4 -0.661813 ... -0.781710 0.352598 0.777792 188\n4 4 1 5 -0.621816 ... -0.136018 0.463253 1.059552 187\n... ... ... ... ... ... ... ... ... ...\n20626 20626 100 196 1.618000 ... 2.446751 -1.805173 -2.921113 4\n20627 20627 100 197 1.717992 ... 1.155367 -2.856395 -1.203764 3\n20628 20628 100 198 1.478011 ... 3.092444 -2.081810 -3.292481 2\n20629 20629 100 199 1.098043 ... 1.155367 -2.911722 -2.085072 1\n20630 20630 100 200 2.337940 ... 1.801059 -2.469103 -2.194080 0\n\n[20631 rows x 18 columns], unique_id ds s_2 s_3 ... s_17 s_20 s_21 y\n0 1 193 0.678077 -0.853550 ... -0.781710 0.241943 0.774097 142\n1 1 194 -1.941707 -0.338137 ... -0.136018 1.127183 0.941305 141\n2 1 195 -0.441831 -0.584426 ... -0.136018 1.459148 1.172256 140\n3 1 196 -0.481827 -1.044384 ... -1.427402 1.016528 0.775945 139\n4 1 197 -0.341839 -0.543650 ... -2.073094 0.961200 1.138999 138\n... ... ... ... ... ... ... ... ... ...\n13091 100 394 1.118041 1.456023 ... 0.509675 -0.919933 -0.852719 24\n13092 100 395 1.078044 0.842747 ... 1.155367 -1.362553 -0.116449 23\n13093 100 396 1.518008 0.428459 ... 1.155367 -1.085916 -0.781586 22\n13094 100 397 1.158038 0.728573 ... 1.155367 -0.864606 -0.182962 21\n13095 100 398 0.538089 1.809962 ... 1.801059 -0.643296 -0.962652 20\n\n[13096 rows x 17 columns])",full_data_normalizedY_test_normalizedY_train_normalized


> 2025-09-02 15:33:55,867 [info] Run execution finished: {"name":"preprocessing-input-data","status":"completed"}


In [7]:
preprocessing_run.outputs.keys()

dict_keys(['return', 'Y_train_normalized', 'Y_test_normalized', 'full_data_normalized'])

## Feature engineering

In [8]:
feature_fn = project.get_function("feature")

In [9]:
feature_run = project.run_function(
    function=feature_fn,
    inputs={
        "Y_full_data_normalized": preprocessing_run.outputs["full_data_normalized"],
    },
    local=False,
)

> 2025-09-02 15:33:56,254 [info] Storing function: {"db":"http://mlrun-api:8080","name":"feature-feat-creation","uid":"5013efa3a998435f8dc71f40852bd3c1"}
> 2025-09-02 15:33:56,585 [info] Job is running in the background, pod: feature-feat-creation-kq2rf
> 2025-09-02 15:34:04,062 [info] extracting source from s3://mlrun/predictive-maintenance.zip to /app/code
> 2025-09-02 15:34:05,792 [info] Creating smoothed feature for column: s_21
> 2025-09-02 15:34:05,792 [info] Grouping by: unique_id
> 2025-09-02 15:34:05,792 [info] Using window_length: 15, polyorder: 3
> 2025-09-02 15:34:05,829 [info] Created new feature: s_21_smoothed
> 2025-09-02 15:34:06,243 [info] To track results use the CLI: {"info_cmd":"mlrun get run 5013efa3a998435f8dc71f40852bd3c1 -p predictive-maintenance","logs_cmd":"mlrun logs 5013efa3a998435f8dc71f40852bd3c1 -p predictive-maintenance"}
> 2025-09-02 15:34:06,243 [info] Run execution finished: {"name":"feature-feat-creation","status":"completed"}


project,uid,iter,start,end,state,kind,name,labels,inputs,parameters,results,artifacts
predictive-maintenance,...852bd3c1,0,Sep 02 15:34:04,2025-09-02 15:34:06.231375+00:00,completed,run,feature-feat-creation,kind=jobowner=jovyanmlrun/client_version=1.9.2mlrun/client_python_version=3.11.13host=feature-feat-creation-kq2rf,Y_full_data_normalized,,,Y_full_data_features


> 2025-09-02 15:34:11,071 [info] Run execution finished: {"name":"feature-feat-creation","status":"completed"}


## Train Model

In [10]:
train_fn = project.get_function("train")

In [11]:
train_run = project.run_function(
    function=train_fn,
    inputs={
        "Y_full_data_features": feature_run.outputs["Y_full_data_features"],
    },
    local=False,
)

> 2025-09-02 15:34:11,257 [info] Storing function: {"db":"http://mlrun-api:8080","name":"train-train-model","uid":"d1085f124dbd4a5187e340e4ec431814"}
> 2025-09-02 15:34:11,659 [info] Job is running in the background, pod: train-train-model-h2rn2
> 2025-09-02 15:34:19,104 [info] extracting source from s3://mlrun/predictive-maintenance.zip to /app/code
pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
> 2025-09-02 15:34:27,323 [info] Starting model training with 33727 samples
> 2025-09-02 15:34:27,323 [info] Target column: y
> 2025-09-02 15:34:27,323 [info] Unique ID column: unique_id
> 2025-09-02 15:34:27,324 [info] Time series column: ds
> 2025-09-02 15:34:27,324 [info] Using 15 exogenous variables: ['s_2', 's_3', 's_4', 's_7', 's_8', 's_9', 's_11', 's_12', 's_13', 's_14', 's_15', 's_17', 's_20', 's_21', 's_21

project,uid,iter,start,end,state,kind,name,labels,inputs,parameters,results,artifacts
predictive-maintenance,...ec431814,0,Sep 02 15:34:19,2025-09-02 15:34:40.716370+00:00,completed,run,train-train-model,kind=jobowner=jovyanmlrun/client_version=1.9.2mlrun/client_python_version=3.11.13host=train-train-model-h2rn2,Y_full_data_features,,return=trained_nbeatsx_model,model_summarymodel


> 2025-09-02 15:34:47,076 [info] Run execution finished: {"name":"train-train-model","status":"completed"}


## Predict

In [12]:
prediction_fn = project.get_function("prediction")

In [13]:
prediction_run = project.run_function(
    function=prediction_fn,
    inputs={
        "Y_full": feature_run.outputs["Y_full_data_features"],
        "Y_test_normalized": preprocessing_run.outputs["Y_test_normalized"],
        "model": train_run.outputs['model'],
    },
    local=False,
)

> 2025-09-02 15:34:47,493 [info] Storing function: {"db":"http://mlrun-api:8080","name":"prediction-predict","uid":"d9adc591ef0f476697b1ce19582efccf"}
> 2025-09-02 15:34:47,960 [info] Job is running in the background, pod: prediction-predict-qlgwn
> 2025-09-02 15:34:56,709 [info] extracting source from s3://mlrun/predictive-maintenance.zip to /app/code
pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
> 2025-09-02 15:35:04,262 [info] Starting prediction process...
> 2025-09-02 15:35:04,262 [info] Full data shape: (33727, 18)
> 2025-09-02 15:35:04,262 [info] Test data shape: (13096, 17)
> 2025-09-02 15:35:04,262 [info] Loading trained model...
> 2025-09-02 15:35:04,312 [info] Starting cross-validation with 10 windows...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: Fa

project,uid,iter,start,end,state,kind,name,labels,inputs,parameters,results,artifacts
predictive-maintenance,...582efccf,0,Sep 02 15:34:56,2025-09-02 15:35:14.262149+00:00,completed,run,prediction-predict,kind=jobowner=jovyanmlrun/client_version=1.9.2mlrun/client_python_version=3.11.13host=prediction-predict-qlgwn,Y_fullY_test_normalizedmodel,,,predictionsprediction_metrics


> 2025-09-02 15:35:18,794 [info] Run execution finished: {"name":"prediction-predict","status":"completed"}
